In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from Backtest.portfolio import PortfolioHandler
from MOMStrategy import MOMStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import hyperopt.pyll.stochastic

In [3]:
def run_backtest(config, trading_data, ohlc_data, window, num, rev):
    window = int(window)
    num = int(num)
    config['title'] = "MOMStrategy" + "_" + str(window) + "_" + str(num) + '_' + str(rev)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data=trading_data, ohlc_data=ohlc_data
    )
    portfolio_handler = PortfolioHandler(
        config, data_handler, events_queue
    )
    strategy = MOMStrategy(config, events_queue, data_handler, portfolio_handler,
                         window = window, num = num, rev = rev)  

    backtest = Backtest(config, events_queue, strategy,
                        data_handler=data_handler)

    results = backtest.start_trading()

    # dict_ans = {
    #     "window": window,
    #     "Sharpe Ratio": [results['sharpe']],
    #     "Total Returns": [(results['cum_returns'][-1] - 1)],
    #     "Max Drawdown": [(results["max_drawdown"] * 100.0)],
    #     "Max Drawdown Duration": [(results['max_drawdown_duration'])],
    #     "Trades": [results['trade_info']['trading_num']],
    #     "Trade Winning": [results['trade_info']['win_pct']],
    #     "Average Trade": [results['trade_info']['avg_trd_pct']],
    #     "Average Win": [results['trade_info']['avg_win_pct']],
    #     "Average Loss": [results['trade_info']['avg_loss_pct']],
    #     "Best Trade": [results['trade_info']['max_win_pct']],
    #     "Worst Trade": [results['trade_info']['max_loss_pct']],
    #     "Worst Trade Date": [results['trade_info']['max_loss_dt']],
    #     "Avg Days in Trade": [results['trade_info']['avg_dit']]
    # }
    # return pd.DataFrame(dict_ans)
    return -1 * (results['cum_returns'][-1] - 1)

In [4]:
def f(params):
#     config = params['config']
#     ohlc_data = params['ohlc_data']
#     trading_data = params['trading_data']
    config = {
        "csv_dir": "C:/backtest/Binance",
        "out_dir": "C:/backtest/results/MOMStrategy",
        "title": "MOMStrategy",
        "is_plot": False,
        "save_plot": False,
        "save_tradelog": False,
        "start_date": pd.Timestamp("2018-01-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
        "end_date": pd.Timestamp("2018-04-01T00:00:00", freq="60" + "T"),
        "equity": 1.0,
        "freq": 60,  # min
        "commission_ratio": 0.002,
        "suggested_quantity": None,     # None or a value
        "max_quantity": None,           # None or a value, Maximum purchase quantity
        "min_quantity": None,           # None or a value, Minimum purchase quantity
        "min_handheld_cash": None,      # None or a value, Minimum handheld funds
        "exchange": "Binance",
        "tickers": ['BTCUSDT', 'CMTBNB', 'CMTBTC', 'CMTETH', 
                    'ETHUSDT', 'LTCUSDT', 'VENBNB', #  'EOSUSDT' 'XRPUSDT'
                   'VENBTC', 'VENETH']
    }
    # trading_data = {}
    # for ticker in config['tickers']:
    #     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
    #     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

    ohlc_data = {}
    for ticker in config['tickers']:
        # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
        ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '_OHLC_60min.h5', key=ticker)

    trading_data = None
    
    window = params['window']
    num = params['num']
    rev = 1
    val = run_backtest(config, trading_data, ohlc_data, window = window, num = num, rev = rev)
    return {'loss': val, 'status': STATUS_OK}

In [6]:
space = {
#     'config': hp.choice('config', [config]),
#     'ohlc_data': hp.choice('ohlc_data', [ohlc_data]),
#     'trading_data': hp.choice('trading_data', [trading_data]),
    'window': hp.randint('window', 240),
    'num': hp.choice('num', tuple(np.arange(2, 6, 1)))
}

In [7]:
# print(hyperopt.pyll.stochastic.sample(space))

In [8]:
trials = Trials()

In [9]:
best = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

---------------------------------
MOMStrategy_228_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Ru

Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4457862900
Sortino Ratio: -2.5056564427
Max Drawdown: 0.7543226599
Max Drawdown Duration: 208800
Total Returns: -0.5292523177
Annualized Returns: -2.1464121772
Compound Annual Growth Rate: -0.9529050513
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: na

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -2.3043540227
Sortino Ratio: -3.3404025695
Max Drawdown: 0.7625810126
Max Drawdown Duration: 208800
Total Returns: -0.6166287577
Annualized Returns: -2.5007721840
Compound Annual Growth Rate: -0.9795192489
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
---------------------------------
MOMStrategy_4_3_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04

Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4457862900
Sortino Ratio: -2.5056564427
Max Drawdown: 0.7543226599
Max Drawdown Duration: 208800
Total Returns: -0.5292523177
Annualized Returns: -2.1464121772
Compound Annual Growth Rate: -0.9529050513
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
---------------------------------
MOMStrategy_7_2_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4777464194
Sortino Ratio: -2.5256294681
Max Drawdown: 0.7199265031
Max Drawdown Duration: 208800
Total Returns: -0.4863381289
Annualized Returns: -1.9723713006
Compound Annual Growth Rate: -0.9329133917
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
---------------------------------
MOMStrategy_77_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-0

...

Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.2197888050
Sortino Ratio: -2.1537001569
Max Drawdown: 0.7607232266
Max Drawdown Duration: 208800
Total Returns: -0.4934362534
Annualized Returns: -2.0011581388
Compound Annual Growth Rate: -0.9365944601
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
-------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4777464194
Sortino Ratio: -2.5256294681
Max Drawdown: 0.7199265031
Max Drawdown Duration: 208800
Total Returns: -0.4863381289
Annualized Returns: -1.9723713006
Compound Annual Growth Rate: -0.9329133917
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
---------------------------------
MOMStrategy_45_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-0

Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4777464194
Sortino Ratio: -2.5256294681
Max Drawdown: 0.7199265031
Max Drawdown Duration: 208800
Total Returns: -0.4863381289
A

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4777464194
Sortino Ratio: -2.5256294681
Max Drawdown: 0.7199265031
Max Drawdown Duration: 208800
Total Returns: -0.4863381289
Annualized Returns: -1.9723713006
Compound Annual Growth Rate: -0.9329133917
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
---------------------------------
MOMStrategy_103_5_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-

Data Time Interval for CMTBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for CMTETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for ETHUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for LTCUSDT:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBNB:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENBTC:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
Data Time Interval for VENETH:
	Start Date	: 2018-01-01 00:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -1.4457862900
Sortino Ratio: -2.5

In [10]:
best

{'num': 3, 'window': 97}

In [12]:
for trial in trials.trials:
    print(trial['misc']['vals'], "\t",trial['result']['loss'])

{'num': [1], 'window': [228]} 	 0.5292523176543278
{'num': [1], 'window': [96]} 	 0.5292523176543278
{'num': [2], 'window': [228]} 	 0.49343625339226105
{'num': [3], 'window': [97]} 	 0.4863381289060521
{'num': [3], 'window': [27]} 	 0.4863381289060521
{'num': [1], 'window': [210]} 	 0.5292523176543278
{'num': [2], 'window': [181]} 	 0.49343625339226105
{'num': [0], 'window': [140]} 	 0.6166287576963398
{'num': [1], 'window': [112]} 	 0.5292523176543278
{'num': [1], 'window': [40]} 	 0.5292523176543278
{'num': [0], 'window': [218]} 	 0.6166287576963398
{'num': [1], 'window': [4]} 	 0.5292523176543278
{'num': [0], 'window': [138]} 	 0.6166287576963398
{'num': [2], 'window': [16]} 	 0.49343625339226105
{'num': [2], 'window': [154]} 	 0.49343625339226105
{'num': [2], 'window': [215]} 	 0.49343625339226105
{'num': [1], 'window': [96]} 	 0.5292523176543278
{'num': [0], 'window': [7]} 	 0.6166287576963398
{'num': [3], 'window': [135]} 	 0.4863381289060521
{'num': [0], 'window': [100]} 	 0.61